#  Intro to Data Visualization with Python-Part1

###  Importing libraries

In [3]:
# Import libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [4]:
path=r'C:\Users\61491\12-2020 Instacart Basket Analysis'

In [5]:
df_customers = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'))

In [6]:
df_customers.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [7]:
df_customers.shape

(206209, 10)

### Data wrangling

In [11]:
# Renaming columns
df_customers.rename(columns= {'First Name' : 'first_name', 'Surnam' : 'surname','Gender' : 'gender','STATE' : 'state',
                          'Age' : 'age', 'n_dependants': 'no_of_dependants', 'fam_status' : 'marital_status'}, inplace = True)

In [12]:
df_customers.rename(columns={'user_id' : 'customer_id'}, inplace = True)

In [13]:
df_customers.head()

,customer_id,first_name,surname,gender,state,age,date_joined,no_of_dependants,marital_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


### Data consistency check

In [14]:
# Checking for missing values
df_customers.isnull().sum()

customer_id             0
first_name          11259
surname                 0
gender                  0
state                   0
age                     0
date_joined             0
no_of_dependants        0
marital_status          0
income                  0
dtype: int64

In [15]:
# View missing values
df_customers[df_customers['first_name'].isnull()==True]

,customer_id,first_name,surname,gender,state,age,date_joined,no_of_dependants,marital_status,income
53,76659,NaN,Gilbert,Male,Colorado,26,1/1/2017,2,married,41709
73,13738,NaN,Frost,Female,Louisiana,39,1/1/2017,0,single,82518
82,89996,NaN,Dawson,Female,Oregon,52,1/1/2017,3,married,117099
99,96166,NaN,Oconnor,Male,Oklahoma,51,1/1/2017,1,married,155673
105,29778,NaN,Dawson,Female,Utah,63,1/1/2017,3,married,151819
...,...,...,...,...,...,...,...,...,...,...
206038,121317,NaN,Melton,Male,Pennsylvania,28,3/31/2020,3,married,87783
206044,200799,NaN,Copeland,Female,Hawaii,52,4/1/2020,2,married,108488
206090,167394,NaN,Frost,Female,Hawaii,61,4/1/2020,1,married,45275
206162,187532,NaN,Floyd,Female,California,39,4/1/2020,0,single,56325


The first_name missing values wont be fixed since its string data.

In [16]:
# Checking for duplicates
df_customers[df_customers.duplicated()]

,customer_id,first_name,surname,gender,state,age,date_joined,no_of_dependants,marital_status,income


There are no duplicates in the customer data

In [17]:
# Checking for mixed data types
for col in df_customers.columns.tolist():
  weird = (df_customers[[col]].applymap(type) != df_customers[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_customers[weird]) > 0:
    print (col)

first_name


In [18]:
# Check first_name data type 
df_customers['first_name'].dtype

dtype('O')

In [19]:
# Convert first_name data type to string
df_customers['first_name'] = df_customers['first_name'].astype('str')

In [21]:
# Re-Check mixed data types
for col in df_customers.columns.tolist():
  weird = (df_customers[[col]].applymap(type) != df_customers[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_customers[weird]) > 0:
    print (col)

There are no more mixed data types

### Merge data 

In [24]:
# Import orders_products_merged pkl
df_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged_update2.pkl'))

In [25]:
df_merge.head()

,order_id,customer_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,Busiest_day,Busiest_days,Slowest_days,Busiest_period_of_day,max_order,loyalty_flag,avg_price,spender_flag,median_days_since_prior_order,frequency_flag
0,94891,4,4,5,13,15.0,22199,1,0,Extra-Dry Cava,...,Regularly busy,Regularly busy,Regularly busy,Most orders,4,New customer,8.650000,Low spender,15.0,Regular customer
1,31925,47329,4,3,10,12.0,22199,1,0,Extra-Dry Cava,...,Regularly busy,Slowest days,Slowest days,Most orders,4,New customer,9.200000,Low spender,12.0,Regular customer
2,94891,4,4,5,13,15.0,25146,2,0,Original Orange Juice,...,Regularly busy,Regularly busy,Regularly busy,Most orders,4,New customer,8.650000,Low spender,15.0,Regular customer
3,95113,410,2,1,18,7.0,25146,18,0,Original Orange Juice,...,Regularly busy,Busiest days,Busiest days,Average orders,28,Regular customer,6.588889,Low spender,7.0,Frequent customer
4,109354,658,14,0,15,17.0,25146,22,0,Original Orange Juice,...,Busiest day,Busiest days,Busiest days,Most orders,14,Regular customer,7.604000,Low spender,17.0,Regular customer


In [26]:
df_merge.shape

(1047658, 25)

In [28]:
# Merge
df_merged = df_merge.merge(df_customers, on = 'customer_id')

In [29]:
df_merged.head()

,order_id,customer_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,frequency_flag,first_name,surname,gender,state,age,date_joined,no_of_dependants,marital_status,income
0,94891,4,4,5,13,15.0,22199,1,0,Extra-Dry Cava,...,Regular customer,Bobby,Reed,Male,Arkansas,31,9/21/2017,3,married,45435
1,94891,4,4,5,13,15.0,25146,2,0,Original Orange Juice,...,Regular customer,Bobby,Reed,Male,Arkansas,31,9/21/2017,3,married,45435
2,31925,47329,4,3,10,12.0,22199,1,0,Extra-Dry Cava,...,Regular customer,Carl,Bridges,Male,Alabama,49,6/25/2017,0,single,159539
3,31925,47329,4,3,10,12.0,24968,2,1,Sonoma Brut Sparkling Wine,...,Regular customer,Carl,Bridges,Male,Alabama,49,6/25/2017,0,single,159539
4,95113,410,2,1,18,7.0,25146,18,0,Original Orange Juice,...,Frequent customer,Stephanie,Saunders,Female,Alaska,69,3/17/2020,3,married,73977


In [30]:
df_merged['_merge'].value_counts()

both          1047658
left_only           0
right_only          0
Name: _merge, dtype: int64

In [31]:
df_merged.shape

(1047658, 34)

#### Export your dataframe as a pickle file and store it correctly in your “Prepared Data” folder.

In [32]:
# Exporting as pkl
df_merged.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'df_merged_final.pkl'))

In [33]:
# Exporting as 'csv'
df_merged.to_csv(os.path.join(path, '02 Data', 'Prepared Data', 'df_merged_final.csv'))